In [21]:
from sklearn.preprocessing import PolynomialFeatures, FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression
from sklearn.model_selection import GridSearchCV
from feature_engine.discretisation import ArbitraryDiscretiser
import math
import pandas as pd
import numpy as np


df = pd.read_csv('movies_clean_v3.csv')
def assign_genre_weight(row):
    # Vérifier si row est une chaîne de caractères
    if isinstance(row, str):
        genres = row.split(',')  # Crée une liste de genres à partir de la chaîne
        if 'Comédie' in genres:  # Vérifie si "Comédie" est dans la liste des genres
            return 3
        elif 'Action' in genres or 'Thriller' in genres:  # Vérifie la présence de "Action" ou "Thriller"
            return 2
        else:
            return 1
    else:
        # Retourner un poids par défaut pour les valeurs NaN ou non-chaînes
        return 0  # ou une autre valeur appropriée selon votre logique d'analyse

df['genre_weight'] = df['genre'].apply(assign_genre_weight)
print(df.head(), df.shape)
# separation des features et de la variable cible
X = df.drop('entrees_premiere_semaine', axis=1)
y = df[['entrees_premiere_semaine']]
print(f'''verif des dimensions X et Y
      X (dataset sans la variable cible): {X.shape}
      Y (la variable cible) : {y.shape}''')


# division du dataset en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42)
print(f''' verif du split 80 20 
80% du dataset : X train -> {X_train.shape}, Y train -> {y_train.shape}
20% du dataset : X test -> {X_test.shape}, Y test -> {y_test.shape}''')

                                             acteurs     budget  \
0                  Sam Worthington, Sigourney Weaver  315000000   
1  Daniel Radcliffe, Emma Watson, Gary Oldman, Al...  130000000   
2  Adam Driver, Benicio Del Toro, Joseph Gordon-L...  200000000   
3  Chris Evans, Chris Hemsworth, Josh Brolin, Rob...  295000000   
4    Sam Worthington, Sigourney Weaver, Kate Winslet  350000000   

      compositeur        date  entrees_premiere_semaine  franchise  \
0    James Horner  2009-12-16                   2648596  Franchise   
1   John Williams  2004-06-02                   2503699  Franchise   
2   John Williams  2017-12-13                   2510462  Franchise   
3  Alan Silvestri  2018-04-25                   2565953  Franchise   
4             NaN  2022-12-14                   2739848  Franchise   

             genre        pays        producteur       realisateur remake  \
0  Science Fiction  Etats-Unis               NaN     James Cameron    NaN   
1          Fantasy  Et

In [22]:
# Sélection des variables catégorielles et numériques
cat_vars = X.select_dtypes(include=['object']).columns.tolist()
num_vars = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_vars.remove('date') 
cat_vars.remove('titre') 
num_vars.remove('scoring_acteurs') 
num_vars,cat_vars


(['budget', 'coeff_studio', 'scoring_acteurs&realisateur', 'genre_weight'],
 ['acteurs',
  'compositeur',
  'franchise',
  'genre',
  'pays',
  'producteur',
  'realisateur',
  'remake',
  'studio',
  'season'])

In [23]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, FunctionTransformer, OneHotEncoder
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np
from sklearn.impute import SimpleImputer
# function custom
def log_transform(x):
    return np.log(x + 1)

log_transformer = FunctionTransformer(log_transform)

#processeur pour les colonnes numériques 
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('log', FunctionTransformer(log_transform)),
  


])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
    
])

# Préparation du préprocesseur
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_vars),
        ('cat', categorical_transformer, cat_vars)
    ])

# Modèle de régression
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', RandomForestRegressor(n_estimators=100, random_state=0))])

# Séparation des données
X = df.drop('entrees_premiere_semaine', axis=1)
y = df['entrees_premiere_semaine']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Entraînement du modèle
model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluation du modèle
mae = mean_absolute_error(y_test, y_pred)

mae
# # Fonction personnalisée pour la transformation logarithmique
# def log_transform(x):
#     return np.log(x + 1)

# log_transformer = FunctionTransformer(log_transform, validate=False)


# # Processeur pour les colonnes numériques
# preprocessor_num = Pipeline(steps=[
#     ('log', log_transformer),
#     # Ajoutez ici d'autres transformations numériques si nécessaire
# ])

# # Processeur pour les colonnes catégorielles
# preprocessor_cat = Pipeline(steps=[
#     ('encoder', OneHotEncoder(sparse_output=True))  # Conserve les sorties comme matrices creuses
# ])

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', preprocessor_num, num_vars),
#         ('cat', preprocessor_cat, cat_vars)
#     ]
# )

# pipeline_elasticnet = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('poly', PolynomialFeatures(degree=2, include_bias=False)),
#     ('scaler', StandardScaler(with_mean=False)),  # Modifié pour gérer les matrices creuses
#     ('regression', ElasticNet())
# ])

# # Test de différents hyperparamètres pour ElasticNet
# param_grid_elasticnet = {
#     'regression__alpha': np.arange(0.01, 5, 0.1),
#     'regression__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9, 1]
# }

# ### Grid search
# grid_elasticnet = GridSearchCV(pipeline_elasticnet, param_grid_elasticnet, cv=10)

# # Entraîner le pipeline sur les données d'entraînement
# grid_elasticnet.fit(X_train, y_train)

# best_model = grid_elasticnet.best_estimator_
# # Prédire y sur l'ensemble de test avec le même pipeline
# y_pred = best_model.predict(X_test)


142242.7341327466

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Suppression des colonnes non nécessaires et prétraitement
df = df.drop(['acteurs', 'scoring_acteurs'], axis=1)

# Gestion des valeurs manquantes pour les colonnes numériques et catégorielles
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.drop('entrees_premiere_semaine')
numeric_transformer = SimpleImputer(strategy='median')

categorical_features = df.select_dtypes(include=['object']).columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Préparation du préprocesseur
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Modèle de régression
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', RandomForestRegressor(n_estimators=100, random_state=0))])

# Séparation des données
X = df.drop('entrees_premiere_semaine', axis=1)
y = df['entrees_premiere_semaine']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Entraînement du modèle
model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluation du modèle
mae = mean_absolute_error(y_test, y_pred)

mae


142204.8485368537

In [25]:
#test des les differents metriques sur modele lasso
#comparaison du y prédit avec le y de test
mse = mean_squared_error(y_test, y_pred)


r2 = r2_score(y_test, y_pred)

print(f'Coefficient de determination R² du modèle elasticnet: {r2}')

rmse = math.sqrt(mse)
print(f'Root Mean Squared Error (RMSE) du modèle elasticnet: {rmse}')

Coefficient de determination R² du modèle elasticnet: 0.5748933710123321
Root Mean Squared Error (RMSE) du modèle elasticnet: 278248.66125159257
